In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl
//spark.sparkContext.version

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@14307d03


Some(http://iuni2:4040)

In [2]:
val WoS = spark.read.format("parquet").load("/WoSraw/WoS2019/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name","static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword",
                      "static_data.fullrecord_metadata.references.reference",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack.grants.grant")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 14 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]

In [3]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

//WoS2.describe().filter($"summary" === "count").show
val cityJ = WoS2.select("UID","titles","pub_info._pubyear","address_name.address_spec.organizations.organization","address_name.address_spec.city","address_name.address_spec.state","address_name.address_spec.country","address_name.address_spec.full_address")//.filter("city is not null")
val cityJ2 = cityJ.withColumn("titles", explode(col("titles.title")))
cityJ2.groupBy("titles._type").count().show

+--------------+--------+
|         _type|   count|
+--------------+--------+
|     abbrev_11|71493700|
|   book_series| 4911983|
|book_seriessub|    3249|
| source_abbrev|71493278|
|        source|74883485|
| book_subtitle|  468957|
|    abbrev_iso|61763329|
|        series| 4912042|
|          item|74883956|
|     abbrev_29|71493199|
|          book|   61966|
+--------------+--------+



cityJ = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 6 more fields]
cityJ2 = [UID: string, titles: struct<_VALUE: string, _translated: string ... 1 more field> ... 6 more fields]


[UID: string, titles: struct<_VALUE: string, _translated: string ... 1 more field> ... 6 more fields]

In [4]:
val InstList = cityJ.withColumn("Institutions", explode(col("organization"))).cache()
InstList.printSchema

root
 |-- UID: string (nullable = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- _pubyear: long (nullable = true)
 |-- organization: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _pref: string (nullable = true)
 |-- city: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- country: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- full_address: array (nullable = true)
 |    |-- element: string (containsNull = 

InstList = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]

In [5]:
val UIDinstEnhanced = InstList.select("UID","_pubyear","full_address","city","state","country","Institutions._VALUE","Institutions._pref").filter(array_contains($"_pref","Y"))
UIDinstEnhanced.describe().filter($"summary" === "count").show

+-------+--------+--------+
|summary|     UID|_pubyear|
+-------+--------+--------+
|  count|94116225|94116225|
+-------+--------+--------+



UIDinstEnhanced = [UID: string, _pubyear: bigint ... 6 more fields]


[UID: string, _pubyear: bigint ... 6 more fields]

In [6]:
InstList.describe().filter($"summary" === "count").show
//UIDinstEnhanced.groupBy("_pubyear").count().orderBy("_pubyear").show(120)

+-------+---------+---------+
|summary|      UID| _pubyear|
+-------+---------+---------+
|  count|110470160|110470160|
+-------+---------+---------+



In [7]:
import org.apache.spark.sql.SaveMode
UIDinstEnhanced.write.mode(SaveMode.Overwrite)
.parquet("/data1/WoSinstEnhanced/")

In [ ]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

val UIDinstEnhanced = spark.read.format("parquet").load("/data1/WoSinstEnhanced/*.parquet")

UIDinstEnhanced.printSchema

In [8]:
val UIDinstEnhanced2 = UIDinstEnhanced.withColumn("institution", array_join($"_VALUE", ";"))
                                      .withColumn("full_addresses", array_join($"full_address", "|"))
                                      .withColumn("cities", array_join($"city", "|"))
                                      .withColumn("states", array_join($"state", "|"))
                                      .withColumn("countries", array_join($"country", "|"))
UIDinstEnhanced2.filter($"UID" === "WOS:000257661900001").show(false)

+-------------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------+--

UIDinstEnhanced2 = [UID: string, _pubyear: bigint ... 11 more fields]


[UID: string, _pubyear: bigint ... 11 more fields]

In [43]:
val instEnhanced = UIDinstEnhanced2.groupBy("_VALUE").agg(concat_ws(";", collect_list("UID")) as "paperList",
                                                         concat_ws(";", collect_list("full_addresses")) as "full_addresses",
                                                         concat_ws(";", collect_list("cities")) as "cities",
                                                         concat_ws(";", collect_list("states")) as "states",
                                                         concat_ws(";", collect_list("countries")) as "countries",
                                                         countDistinct("UID") as "paperCount").cache()
instEnhanced.describe().filter($"summary" === "count").show
//distribution by paper counts per institution, and fitler the lower ones , rough estimate 20k
//compare city, state, country with instEnhanced
//check see if large institiutions over mulitple sites like Max Planck Inst, Chinese Acad Med Sci, POLISH ACAD SCI, Polish Academy of Sciences, see how institution enhancement is done

Name: org.apache.spark.SparkException
Message: Job 78 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1890)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1825)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2083)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2060)


In [34]:
instEnhanced.filter(array_contains($"_VALUE","Polish Acad")).show()

+--------------------+--------------------+--------------------+--------------------+------+--------------------+----------+
|              _VALUE|           paperList|      full_addresses|              cities|states|           countries|paperCount|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+----------+
|[Polish Acad, Pol...| WOS:000257197001284|Ctr Med Expt, Ins...|       Prague|Warsaw|      |Czech Republic|Po...|         1|
|[Polish Acad, Hir...|WOS:0002790513021...|Polish Acad, Ludw...|Wroclaw|Wroclaw|W...|     ;|Poland|Poland|Pol...|         2|
+--------------------+--------------------+--------------------+--------------------+------+--------------------+----------+



lastException: Throwable = null


In [24]:
val instCompact = UIDinstEnhanced2.groupBy("_VALUE").agg(concat_ws(";", collect_list(distinct "UID")) as "paperList",
                                                         countDistinct("UID") as "paperCount")
                                                    .withColumn("institution", array_join($"_VALUE", ";")).cache()
instCompact.describe().filter($"summary" === "count").show

+-------+---------+----------+-----------+
|summary|paperList|paperCount|institution|
+-------+---------+----------+-----------+
|  count|   968957|    968957|     968957|
+-------+---------+----------+-----------+



instCompact = [_VALUE: array<string>, paperList: string ... 2 more fields]


[_VALUE: array<string>, paperList: string ... 2 more fields]

In [26]:
instCompact.filter($"institution".contains("Max Planck")).orderBy($"paperCount".desc).show(50)

+--------------------+--------------------+----------+--------------------+
|              _VALUE|           paperList|paperCount|         institution|
+--------------------+--------------------+----------+--------------------+
|[Max Planck Inst ...|WOS:0000855248000...|      9248|Max Planck Inst E...|
|[Max Planck Inst ...|WOS:0001699436000...|      7317|Max Planck Inst A...|
|[Max Planck Inst ...|WOS:0001756380000...|      7288|Max Planck Inst P...|
|[Max Planck Inst ...|WOS:0001689318000...|      6954|Max Planck Inst B...|
|[Max Planck Inst ...|WOS:0001722977000...|      6539|Max Planck Inst B...|
|[Max Planck Inst ...|WOS:0000864894000...|      6084|Max Planck Inst K...|
|[Max Planck Inst ...|WOS:0001694093000...|      5920|Max Planck Inst F...|
|[Max Planck Inst ...|WOS:0001767524000...|      5878|Max Planck Inst A...|
|[Max Planck Inst ...|WOS:0001665096000...|      5748|Max Planck Inst C...|
|[Max Planck Inst ...|WOS:0001656460000...|      5248|Max Planck Inst P...|
|[Max Planck

In [28]:
val instFiltered = instCompact.filter($"paperCount" > 2000).orderBy($"paperCount".desc)
instFiltered.describe().filter($"summary" === "count").show

+-------+---------+----------+-----------+
|summary|paperList|paperCount|institution|
+-------+---------+----------+-----------+
|  count|     5805|      5805|       5805|
+-------+---------+----------+-----------+



instFiltered = [_VALUE: array<string>, paperList: string ... 2 more fields]


[_VALUE: array<string>, paperList: string ... 2 more fields]

In [32]:
import org.apache.spark.sql.SaveMode
instFiltered.select("institution","paperCount","paperList").write.mode(SaveMode.Overwrite)
.parquet("/data1/WoSinstEnhanced/instCompact/")

In [31]:
instFiltered.select("institution","paperCount").coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")//.option("compression", "gzip")
    .save("/data1/WoSinstEnhanced/instCompact/CSV/")

In [35]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}

val cityJ3 = cityJ.withColumn("journal", when(col("titles.title").isNotNull, getTitle("source")($"titles.title")).otherwise(lit(null)))
cityJ3.show

+-------------------+--------------------+--------+--------------------+-------------+---------+-------------+--------------------+--------------------+
|                UID|              titles|_pubyear|        organization|         city|    state|      country|        full_address|             journal|
+-------------------+--------------------+--------+--------------------+-------------+---------+-------------+--------------------+--------------------+
|WOS:A1985AUK0600019|[6, [[CLINICA CHI...|    1985|                null|         null|     null|         null|                null|CLINICA CHIMICA ACTA|
|WOS:A1985ATW9400035|[6, [[INSULA-REVI...|    1985|                null|         null|     null|         null|                null|INSULA-REVISTA DE...|
|WOS:A1985AHR7600027|[6, [[ELECTRONICS...|    1985|                null|         null|     null|         null|                null| ELECTRONICS LETTERS|
|WOS:A1985AJM7300014|[5, [[ACTA MORPHO...|    1985|[[[STATE UNIV UTR...|    [UTREC

cityJ3 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]


getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]

In [36]:
val cols = cityJ3.select("UID", "city", "state", "country").columns.filter(_ != "UID")

val cityJ4 = cols.
  foldLeft(cityJ3)( (accDF, c) => 
    accDF.withColumn(c, struct(lit(c).as("name"), col(c).as("value")))
  ).
  select($"UID", array(cols.map(col): _*).as("cityFull"))

cityJ4.show(false)

+-------------------+-------------------------------------------------------------+
|UID                |cityFull                                                     |
+-------------------+-------------------------------------------------------------+
|WOS:A1985AUK0600019|[[city,], [state,], [country,]]                              |
|WOS:A1985ATW9400035|[[city,], [state,], [country,]]                              |
|WOS:A1985AHR7600027|[[city,], [state,], [country,]]                              |
|WOS:A1985AJM7300014|[[city, [UTRECHT]], [state, []], [country, [NETHERLANDS]]]   |
|WOS:A1985APH5500026|[[city,], [state,], [country,]]                              |
|WOS:A1985AAK3200005|[[city,], [state,], [country,]]                              |
|WOS:A1985AVL2400004|[[city,], [state,], [country,]]                              |
|WOS:A1985AVR2500014|[[city, [CAMBRIDGE]], [state, []], [country, [ENGLAND]]]     |
|WOS:A1985APD6100014|[[city,], [state,], [country,]]                        

cols = Array(city, state, country)
cityJ4 = [UID: string, cityFull: array<struct<name:string,value:array<string>>>]


[UID: string, cityFull: array<struct<name:string,value:array<string>>>]

In [37]:
import org.apache.spark.sql.functions.arrays_zip

val cityJ5 = cityJ3.withColumn(
  "cityFull", arrays_zip($"city", $"state", $"country")
)

cityJ5.show(false)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------+-------------+-------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------------------------+
|UID                |titles                                                                                                                                                                                                                              

cityJ5 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 8 more fields]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 8 more fields]

In [38]:
import org.apache.spark.sql.functions.{concat, lit}

val cityJ6 = cityJ5.select($"UID",$"journal",explode($"cityFull")).select("UID","journal","col.city","col.state","col.country").na.fill("")
val cityJ7 = cityJ6.withColumn(
  "cityFull", concat($"city", lit(" "), $"state", lit(" "), $"country")
)
//val cityJ8 = cityJ7.groupBy("UID").agg(collect_list("cityFull") as "cityFullList", first("journal") as "journal")
//cityJ8.show(100,false)

cityJ6 = [UID: string, journal: string ... 3 more fields]
cityJ7 = [UID: string, journal: string ... 4 more fields]


[UID: string, journal: string ... 4 more fields]

In [45]:
cityJ7.filter($"UID" === "WOS:000257661900001").show(false)

+-------------------+-----------------------------+--------------+-----+-------+----------------------+
|UID                |journal                      |city          |state|country|cityFull              |
+-------------------+-----------------------------+--------------+-----+-------+----------------------+
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Pasadena      |CA   |USA    |Pasadena CA USA       |
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Socorro       |NM   |USA    |Socorro NM USA        |
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Cambridge     |MA   |USA    |Cambridge MA USA      |
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Gif Sur Yvette|     |France |Gif Sur Yvette  France|
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Heidelberg    |     |Germany|Heidelberg  Germany   |
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Amherst       |MA   |USA    |Amherst MA USA        |
|WOS:000257661900001|ASTROPHYSICAL JOURNAL LETTERS|Honolulu     

In [39]:
import org.apache.spark.sql.SaveMode

val cityJ9 = cityJ7.groupBy("cityFull").agg(count(lit(1)).alias("paperCount"))
cityJ9.orderBy($"paperCount".desc).show(50,false)
//cityJ9.write.mode(SaveMode.Overwrite).parquet("/user/yan30/WoScityJournal/")

+-----------------------------------+----------+
|cityFull                           |paperCount|
+-----------------------------------+----------+
|Beijing  Peoples R China           |1741970   |
|London  England                    |1299390   |
|Boston MA USA                      |1061472   |
|New York NY USA                    |1023663   |
|Tokyo  Japan                       |949282    |
|Seoul  South Korea                 |939495    |
|Paris  France                      |862381    |
|Shanghai  Peoples R China          |799271    |
|Toronto ON Canada                  |631233    |
|Moscow  Russia                     |577859    |
|Los Angeles CA USA                 |564685    |
|Madrid  Spain                      |559716    |
|Philadelphia PA USA                |548318    |
|Baltimore MD USA                   |528832    |
|Chicago IL USA                     |523843    |
|Taipei  Taiwan                     |511066    |
|Houston TX USA                     |503769    |
|Rome  Italy        

cityJ9 = [cityFull: string, paperCount: bigint]


[cityFull: string, paperCount: bigint]

In [41]:
cityJ9.filter($"paperCount" > 2000).describe().filter($"summary" === "count").show

+-------+--------+----------+
|summary|cityFull|paperCount|
+-------+--------+----------+
|  count|    3790|      3790|
+-------+--------+----------+



lastException: Throwable = null


In [42]:
cityJ9.describe().filter($"summary" === "count").show

+-------+--------+----------+
|summary|cityFull|paperCount|
+-------+--------+----------+
|  count|  524170|    524170|
+-------+--------+----------+



In [ ]:
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.sql.functions.col

val cityJ8 = cityJ6.withColumn(
  "cityFull", regexp_replace(concat($"city", $"state", $"country"),"\\s", "")
)
val cityJ9b = cityJ8.groupBy("cityFull","journal").agg(count(lit(1)).alias("count")).cache()

In [ ]:
//cityJ9b.filter($"cityFull".contains("Nan Kang")).show
//cityJ9b.show(false)
cityJ9b.describe().filter($"summary" === "count").show

In [ ]:
val cityL = spark.read.format("csv").option("header", true).load("/data1/cityList.csv")
val cityList = cityL.withColumn("cityUpper", upper(col("city_rev")));
cityList.show(60)

In [ ]:
val cityM = spark.read.format("csv").option("header", true).load("/data1/cityListMerged.csv")
val cityMerged = cityM.select(" string", " mergeList"," mergeCount").withColumn("mergeLength", length(col(" mergeCount")))
cityMerged.show(60,false)

In [ ]:
val matchedCityMerge = cityMerged.join(cityList, (cityMerged(" mergeList").contains(cityList("city_rev")) 
                                       && cityMerged(" mergeList").contains(cityList("country"))), "inner")
//matchedCityMerge.groupBy("mass").agg(concat_ws(";", collect_list(" matchList")) as "matchLists").show(60)
//val maxMatch = matchedCityMerge.groupBy("city").agg(max("mergeLength") as "mergeMax")
//val cityMatched = maxMatch.join(cityMerged, cityMerged("city")=== maxMatch("city") &&
                                //cityMerged("mergeLength")=== maxMatch("mergeMax"), "left")
//matchedCityMerge.show(60)
//matchedCityMerge.describe().filter($"summary" === "count").show

In [ ]:
matchedCityMerge.filter($"city".contains("Taipei")).show(false)

In [ ]:
import org.apache.spark.sql.expressions.Window
val w = Window.partitionBy($"city")
val maxMatch2 = matchedCityMerge.withColumn("mergeMax", max("mergeLength").over(w))
    .filter($"mergeMax" === $"mergeLength").orderBy("mergeMax")
    .select(" string", " mergeList", "mass", "city", "country", "city_rev")
    .toDF("string", "matchList", "mass", "city", "country", "city_rev")
maxMatch2.show(60)

In [ ]:
val matched0 = maxMatch2.join(cityJ9b, maxMatch2("matchList").contains(cityJ9b("cityFull")), "inner")
//matched.show
matched0.describe().filter($"summary" === "count").show

In [ ]:
val missing = cityList.select("city").except(matched0.select("city"))
//val missing2 = missing.join(accelList2, missing("Name0")===accelList2("Name0"), "inner").toDF()
missing.show(20,false)

In [ ]:
val journals = spark.read.format("csv").option("header", true).load("/data1/journalName.csv")
val journalWoS = journals.filter($"source_type"==="Thomson")
journalWoS.show(false)

In [ ]:
val matched2 = matched0.join(journalWoS, regexp_replace(matched0("journal"),"\\s", "")
                             ===regexp_replace(journalWoS("journal_name"),"\\s", ""), "left").cache()
//matched.show
matched2.describe().filter($"summary" === "count").show

In [ ]:
val JsubMap = spark.read.format("csv").option("header", true).load("/data1/jounral-subdiscipline.csv")
val matched3 = matched2.withColumn("uniqueID",monotonicallyIncreasingId)
                .join(JsubMap, matched2("journ_id")===JsubMap("journ_id"), "left")
matched3.show(false)
//matched3.describe().filter($"summary" === "count").show

In [ ]:
val subTable = spark.read.format("csv").option("header", true).load("/data1/subdiscipline.csv")
val matched4 = matched3.join(subTable, matched3("subd_id")===subTable("subd_id"), "left")
//matched3.show(false)
matched4.describe().filter($"summary" === "count").show

In [ ]:
val subTable = spark.read.format("csv").option("header", true).load("/data1/discipline.csv")
val matched5 = matched4.join(subTable, matched4("disc_id")===subTable("disc_id"), "left")
//matched3.show(false)
matched5.describe().filter($"summary" === "count").show

In [ ]:
matched5.withColumn("jLength", length(col("jfraction"))).orderBy($"jLength".desc).show(60,false)

In [ ]:
val matched5b = matched5.filter($"jfraction".isNotNull).groupBy("uniqueID").pivot("disc_name")
                .agg(sum("jfraction")).na.fill(0)
matched5b.show

In [ ]:
val matched2b = matched2.withColumn("uniqueID",monotonicallyIncreasingId).select("uniqueID","mass","city","country","city_rev","count")
val matched6 = matched2b.join(matched5b, matched2b("uniqueID")===matched5b("uniqueID"), "left").cache()
//matched6.describe().filter($"summary" === "count").show
matched6.show(false)

In [ ]:
val matched7 = matched6.groupBy("city").agg(sum(matched6("Biology")*matched6("count")) as "Biology"
                                           ,sum(matched6("Biotechnology")*matched6("count")) as "Biotechnology"
                                           ,sum(matched6("Brain Research")*matched6("count")) as "Brain Research"
                                           ,sum(matched6("Chemical, Mechanical, & Civil Engineering")*matched6("count")) as "Chemical, Mechanical, & Civil Engineering"
                                           ,sum(matched6("Chemistry")*matched6("count")) as "Chemistry"
                                           ,sum(matched6("Earth Sciences")*matched6("count")) as "Earth Sciences"
                                           ,sum(matched6("Electrical Engineering & Computer Science")*matched6("count")) as "Electrical Engineering & Computer Science"
                                           ,sum(matched6("Health Professionals")*matched6("count")) as "Health Professionals"
                                           ,sum(matched6("Humanities")*matched6("count")) as "Humanities"
                                           ,sum(matched6("Infectious Diseases")*matched6("count")) as "Infectious Diseases"
                                           ,sum(matched6("Math & Physics")*matched6("count")) as "Math & Physics"
                                           ,sum(matched6("Medical Specialties")*matched6("count")) as "Medical Specialties"
                                           ,sum(matched6("Social Sciences")*matched6("count")) as "Social Sciences"
                                           )
matched7.show(55)

In [ ]:
//val exprs = matched6.columns.drop(7).map(sum(_)* matched6.columns.tail(4))
//matched6.groupBy("city").agg(exprs.head, exprs.tail: _*).show()
matched6.groupBy("city").sum(matched6.columns.drop(7): _*).show(55)

In [ ]:
matched6.columns.tail(4)

In [ ]:
matched7.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .save("/tmp/cityProfiles.csv")

In [ ]:
import sqlContext.implicits._
cityJ9b.filter($"cityFull".contains("Munich")).orderBy($"count".desc).show(60, false)